In [2]:
import numpy as np
import pylab as pl
from numpy import fft
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
def main_function_slide_fit_score_slide_range(stock_name,date_start,data_range,slide_range,n_slide,pv_range,n_harm_0,n_harm_1,type):
    predict_range = data_range
    date_start = datetime.datetime.strptime(date_start, '%Y-%m-%d')
    date_start = date_start + relativedelta(months=-data_range)
    df = pd.DataFrame()
    data = pd.DataFrame()
    fit_score = []
    best_fit = []
    df_delay = pd.DataFrame(columns=['Date','delay','pv'])
    fig, axes = plt.subplots(2, 1, figsize=(15, 8))
    for i in range(n_slide) :
        date_predict_start = date_start + relativedelta(months=+data_range)
        date_predict_end = date_predict_start + relativedelta(months=+predict_range)
        data_drop,temp,score_temp,best_fit_temp = main_function_1_fit_score(stock_name,date_start,date_predict_start,date_predict_end,n_harm_0,n_harm_1,pv_range,type)
        fit_score.append(round(score_temp,2))
        best_fit.append(best_fit_temp)
        temp = temp.drop(temp[temp.index < date_predict_start].index)
        # axes[1].plot(temp.index, temp['Close'], label='Predict',linewidth=3)
        temp['peaks'] = peak_valleys(pv_range,temp)[0]
        temp['valleys'] = peak_valleys(pv_range,temp)[1]
        data_temp = yf.Ticker(stock_name).history(start=date_predict_start, end=date_predict_end)
        data_temp['peaks'] = peak_valleys(pv_range,data_temp)[0]
        data_temp['valleys'] = peak_valleys(pv_range,data_temp)[1]
        temp['peaks_delay'] = peak_valleys_delay(data_temp,temp)[0]
        temp['valleys_delay'] = peak_valleys_delay(data_temp,temp)[1]
        # axes[1].plot(temp.index , temp['peaks'],'^',c='royalblue',label='peaks')
        # axes[1].plot(temp.index , temp['valleys'],'v',c='orangered',label='valleys')
        temp = temp.drop(columns='Close')
        # print(temp.dropna(how='all'))
        df_delay.loc[i,'Date'],df_delay.loc[i,'delay'],df_delay.loc[i,'pv'] = get_first_delay(temp)
        df = pd.concat([df[df.index < temp.index[0]],temp])
        date_start = date_start + relativedelta(weeks=+slide_range)
    data = yf.Ticker(stock_name).history(start=date_start, end=date_predict_end)
    axes[0].plot(data.index, data['Close'], 'gray', linewidth=3, label='Test_data')
    data['peaks'] = peak_valleys(pv_range,data)[0]
    data['valleys'] = peak_valleys(pv_range,data)[1]
    axes[0].plot(data.index,data['peaks'],'^',c='royalblue',label='peaks')
    axes[0].plot(data.index,data['valleys'],'v',c='orangered',label='valleys')
    # for i, label in enumerate(df['peaks_delay']):
    #     axes[1].annotate(label, (df['peaks'].index[i],  df['peaks'][i]),fontsize=14)
    # for i, label in enumerate(df['valleys_delay']):
    #     axes[1].annotate(label, (df['valleys'].index[i],  df['valleys'][i]),fontsize=14)
    df_delay = df_delay.set_index(df_delay['Date'])
    df_delay = df_delay.drop(columns='Date')
    axes[0].set_ylabel("price", fontsize=14)
    axes[0].grid(True)
    axes[1].grid(True)
    axes[1].set_ylabel("delay", fontsize=14)

    final_score =  round(sum([abs(ele) for ele in df_delay['delay']]) / len(df_delay['delay']) ,2)
    a = pd.DataFrame(index=df.index,columns=['delay','pv'])
    a.loc[df_delay.index] = df_delay
    axes[0].set_xlim(data.index[0],data.index[-1])
    axes[1].set_xlim(data.index[0],data.index[-1])
    for i, label in enumerate(a['delay']):
        if a['pv'][i] == 'peaks' :
            axes[1].plot(a['delay'].index[i] , a['delay'][i],'*',c='royalblue',label='peaks')
        else :
            axes[1].plot(a['delay'].index[i] , a['delay'][i],'*',c='orangered',label='valleys')
        axes[1].annotate(label, (a['delay'].index[i],  a['delay'][i]),fontsize=14)
    # axes[1].plot(a.index,a['delay'],'*',c='royalblue',label='peaks')
    # axes[0].plot(data.index,data['valleys'],'v',c='orangered',label='valleys')
    plt.show()
    return data,df,fit_score,best_fit,final_score,df_delay